# Triage Classification

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

## Get and Explore Data

In [2]:
df = pd.read_csv("pre-processed.csv")

In [3]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6697 entries, 0 to 6696
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   esi                     6697 non-null   int64  
 1   age                     6697 non-null   int64  
 2   gender                  6697 non-null   object 
 3   bronchitis              6697 non-null   int64  
 4   cardiaarrst             6697 non-null   int64  
 5   cardiacanom             6697 non-null   int64  
 6   carditis                6697 non-null   int64  
 7   chestpain               6697 non-null   int64  
 8   chfnonhp                6697 non-null   int64  
 9   copd                    6697 non-null   int64  
 10  cysticfibro             6697 non-null   int64  
 11  dizziness               6697 non-null   int64  
 12  dysrhythmia             6697 non-null   int64  
 13  fatigue                 6697 non-null   int64  
 14  gastritis               6697 non-null   

In [4]:
df.head()

,esi,age,gender,bronchitis,cardiaarrst,cardiacanom,carditis,chestpain,chfnonhp,copd,...,cc_hypotension,cc_influenza,cc_nausea,cc_numbness,cc_palpitations,cc_rapidheartrate,cc_shortnessofbreath,cc_strokealert,cc_syncope,cc_weakness
0,3,66,Male,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,3,79,Male,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,79,Male,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,80,Male,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,80,Male,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# df.disposition.value_counts()

## Pytorch

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.set_default_dtype(torch.float64)

cuda


In [7]:
# Create custom dataset
class TriageDataset(torch.utils.data.Dataset):
    
    def __init__(self, file_path = "Hospital Triage and Patient History.csv"):
        df = pd.read_csv(file_path)
        
        # Clean data (convert categorical to numeric, remove department name)
        df.drop(columns=['dep_name'], inplace=True)
        for col in df:
            dt = df[col].dtype 
            if dt == int or dt == float:
                df[col].fillna(0, inplace=True)
            else:
                df[col].fillna("", inplace=True)
        
        categorical = []
        for (key, value) in df.dtypes.items():
            if (value == 'object'):
                df[key] = df[key].astype('category')
                categorical.append(key)
                category_num = f'{key}_num'
                df[category_num] = df[key].cat.codes.astype('float64')
            elif (value == 'int64'):
                df[key] = df[key].astype('float64')
        
        df = df.drop(columns=categorical)
        label_ind = df.columns.get_loc('disposition_num')
        features_ind = [i for i in range(df.shape[1])]
        features_ind.remove(label_ind)
        
        # Set features and label
        self.data = df
        self.features = df.iloc[:, features_ind]
        self.label = df.iloc[:, label_ind]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.features.iloc[index].values
        y = self.label.iloc[index]
        return x, y

In [8]:
# Create train and test datasets
triage_dataset = TriageDataset()

generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = torch.utils.data.random_split(triage_dataset, [0.8, 0.2], generator=generator)

# Create dataloaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [9]:
class SmallModel(torch.nn.Module):

    def __init__(self):
        super(SmallModel, self).__init__()

        self.linear1 = torch.nn.Linear(970, 300) # 972 - 2 features
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(300, 100)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(100, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 10)
        self.activation4 = torch.nn.ReLU()
        self.linear5 = torch.nn.Linear(10, 1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        return x


In [10]:
small_model = SmallModel().to(device)
# optimizer = torch.optim.Adam(small_model.parameters(), lr= 0.005, weight_decay= 0.001)
# optimizer = torch.optim.Adam(small_model.parameters(), lr= 0.01, weight_decay= 0.005)
optimizer = torch.optim.Adam(small_model.parameters())

In [11]:
# Training model
def trainer(model, train_loader, test_loader, num_epochs, optimizer):
    train_accs = []
    test_accs = []
    
    for epoch in range(num_epochs):
        model.train()
        
        for i, (features, labels) in enumerate(train_dataloader):
            labels = labels.type(torch.LongTensor)
            features = features.to(device)
            labels = labels.to(device)

            outputs = small_model(features)
            loss = torch.nn.functional.cross_entropy(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 1 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}')

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in train_loader:
                labels = labels.type(torch.LongTensor)
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
        train_accuracy = correct/total
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {train_accuracy:.4f}')
        train_accs.append(train_accuracy)

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in test_loader:
                labels = labels.type(torch.LongTensor)
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_accuracy = correct/total
        print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {test_accuracy:.4f}')
        test_accs.append(test_accuracy)
    
    return train_accs, test_accs

In [12]:
train_accs, test_accs = trainer(small_model, train_dataloader, test_dataloader, 1, optimizer)

Epoch [1/1], Step [1/3504], Loss: 4.6991
Epoch [1/1], Step [2/3504], Loss: 2.0554
Epoch [1/1], Step [3/3504], Loss: 1.6278
Epoch [1/1], Step [4/3504], Loss: 1.2485
Epoch [1/1], Step [5/3504], Loss: 1.0909
Epoch [1/1], Step [6/3504], Loss: 1.1258
Epoch [1/1], Step [7/3504], Loss: 0.8954
Epoch [1/1], Step [8/3504], Loss: 0.8939
Epoch [1/1], Step [9/3504], Loss: 0.7887
Epoch [1/1], Step [10/3504], Loss: 0.7711
Epoch [1/1], Step [11/3504], Loss: 0.8744
Epoch [1/1], Step [12/3504], Loss: 0.7351
Epoch [1/1], Step [13/3504], Loss: 1.0488
Epoch [1/1], Step [14/3504], Loss: 0.7797
Epoch [1/1], Step [15/3504], Loss: 1.5159
Epoch [1/1], Step [16/3504], Loss: 0.7280
Epoch [1/1], Step [17/3504], Loss: 0.6958
Epoch [1/1], Step [18/3504], Loss: 0.7316
Epoch [1/1], Step [19/3504], Loss: 0.7513
Epoch [1/1], Step [20/3504], Loss: 0.6588
Epoch [1/1], Step [21/3504], Loss: 0.6386
Epoch [1/1], Step [22/3504], Loss: 0.6673
Epoch [1/1], Step [23/3504], Loss: 0.5934
Epoch [1/1], Step [24/3504], Loss: 0.6136
E

In [13]:
class PaperModel(torch.nn.Module):

    def __init__(self):
        super(PaperModel, self).__init__()

        self.linear1 = torch.nn.Linear(970, 30) # 972 - 2 features
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(30, 30)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(30, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        return x


In [14]:
paper_model = PaperModel().to(device)
optimizer = torch.optim.RMSprop(paper_model.parameters(), lr=1e-3)

In [15]:
train_accs2, test_accs2 = trainer(paper_model, train_dataloader, test_dataloader, 1, optimizer)

Epoch [1/1], Step [1/3504], Loss: 0.5836
Epoch [1/1], Step [2/3504], Loss: 0.3663
Epoch [1/1], Step [3/3504], Loss: 0.4439
Epoch [1/1], Step [4/3504], Loss: 0.3537
Epoch [1/1], Step [5/3504], Loss: 0.3597
Epoch [1/1], Step [6/3504], Loss: 0.3501
Epoch [1/1], Step [7/3504], Loss: 0.4184
Epoch [1/1], Step [8/3504], Loss: 0.4446
Epoch [1/1], Step [9/3504], Loss: 0.4917
Epoch [1/1], Step [10/3504], Loss: 0.3680
Epoch [1/1], Step [11/3504], Loss: 0.4641
Epoch [1/1], Step [12/3504], Loss: 0.3038
Epoch [1/1], Step [13/3504], Loss: 0.3472
Epoch [1/1], Step [14/3504], Loss: 0.4634
Epoch [1/1], Step [15/3504], Loss: 0.3717
Epoch [1/1], Step [16/3504], Loss: 0.5184
Epoch [1/1], Step [17/3504], Loss: 0.3817
Epoch [1/1], Step [18/3504], Loss: 0.4183
Epoch [1/1], Step [19/3504], Loss: 0.3171
Epoch [1/1], Step [20/3504], Loss: 0.4870
Epoch [1/1], Step [21/3504], Loss: 0.3834
Epoch [1/1], Step [22/3504], Loss: 0.3174
Epoch [1/1], Step [23/3504], Loss: 0.4284
Epoch [1/1], Step [24/3504], Loss: 0.3705
E